# Telecom Churn Case Study Kaggle Competition
## Problem Statement
In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business
goal. To reduce customer churn, telecom companies need to predict which customers are at high risk of churn. In this project, you will analyze customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn.

In this competition, your goal is to build a machine learning model that is able to predict churning customers based on the features provided for their usage.
## Objectives
Objectives
The main goal of the case study is to build ML models to predict churn. The predictive model that you’re going to build will the following purposes:

- It will be used to predict whether a high-value customer will churn or not, in near future (i.e. churn phase). By knowing this, the company can take action steps such as providing special plans, discounts on recharge etc.

- It will be used to identify important variables that are strong predictors of churn. These variables may also indicate why customers choose to switch to other networks.

- Even though overall accuracy will be your primary evaluation metric, you should also mention other metrics like precision, recall, etc. for the different models that can be used for evaluation purposes based on different business objectives. For example, in this problem statement, one business goal can be to build an ML model that identifies customers who'll definitely churn with more accuracy as compared to the ones who'll not churn. Make sure you mention which metric can be used in such scenarios.

- Recommend strategies to manage customer churn based on your observations.

In [ ]:
# Importing Libraries
import pandas as pd 
import numpy as np 

import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.express as px 

from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

from sklearn.pipeline import Pipeline

from imblearn.combine import SMOTETomek

from skopt import BayesSearchCV
from xgboost import XGBClassifier,XGBRFClassifier

import lightgbm as lgb

from sklearn.preprocessing import MinMaxScaler

import scipy.stats as stats

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,precision_recall_curve,PrecisionRecallDisplay,roc_auc_score,roc_curve
import optuna

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Removing display limit of dataframe (optional cell to run)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Setting style for seaonrn
sns.color_palette("seismic", 50)
sns.set_style("whitegrid", {'axes.grid' : False})

In [ ]:
# Importing train dataset and displaying first 5 rows
tel_churn=pd.read_csv('train.csv')
tel_churn.head()

# Importing train dataset and displaying first 5 rows
tel_churn_test=pd.read_csv('test.csv')
tel_churn_test.head()

In [ ]:
# Shape of dataset
tel_churn.shape

In [ ]:
# Information about dataset
tel_churn.info(verbose=1)

In [ ]:
#Basic information about the data
## Number of rows and columns
print('Number of Columns:',tel_churn.shape[1])
print('Number of Rows:',tel_churn.shape[0])
## Number of missing values
print('Number of missing values:',tel_churn.isnull().sum().sum())
## Number of unique values
print('Number of unique values:',tel_churn.nunique().sum())
## Number of duplicates
print('Number of duplicates:',tel_churn.duplicated().sum())

In [ ]:
# Dataset Distribution
tel_churn.describe()

In [ ]:
# Checking Classes Distribution
tel_churn['churn_probability'].value_counts()/tel_churn.shape[0]*100

From describe() and info() we can see that data has large number of null values as well as outliers which needs to be handled. We will handle these issues in the next section.

## Cleaning the data

In [ ]:
# Checking for missing value percentage
pd.DataFrame((tel_churn.isnull().sum()/len(tel_churn)*100).sort_values(ascending=False))

In [ ]:
# Removing column with 30% or more null values as it will reduce the impact on analysis
tel_churn = tel_churn.loc[:,tel_churn.isnull().sum()/tel_churn.shape[0]*100<30]
# Shape of the dataframe after removing columns
tel_churn.shape

In [ ]:
# Checking for missing value percentage
pd.DataFrame((tel_churn.isnull().sum()/len(tel_churn)*100).sort_values(ascending=False))

In [ ]:
# Removing rows with missing values with more than 10 missing values
tel_churn.dropna(axis=0,inplace=True,thresh=tel_churn.shape[1]-10)
# Shape of the dataframe after removing rows
print(tel_churn.shape)

In [ ]:
# List of columns with Date datatype
date_cols = [k for k in tel_churn.columns.to_list() if 'date' in k]
date_cols

In [ ]:
# Converting Date columns to datetime datatype and extracting the days before last day
for i in date_cols:
    tel_churn[i] = pd.to_datetime(tel_churn[i])
    tel_churn[i] = tel_churn[i].dt.date
    tel_churn[i] = pd.to_datetime(tel_churn[i])
    tel_churn[i] = tel_churn[i].dt.daysinmonth - tel_churn[i].dt.day

# for test data set
for i in date_cols:
    tel_churn_test[i] = pd.to_datetime(tel_churn_test[i])
    tel_churn_test[i] = tel_churn_test[i].dt.date
    tel_churn_test[i] = pd.to_datetime(tel_churn_test[i])
    tel_churn_test[i] = tel_churn_test[i].dt.daysinmonth - tel_churn_test[i].dt.day

In [ ]:
# Printing the date columns
tel_churn[date_cols].value_counts()

In [ ]:
# Removing the columns with only one unique value among date columns
tel_churn.drop(columns=['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8'],inplace=True)
date_cols.remove('last_date_of_month_6')
date_cols.remove('last_date_of_month_7')
date_cols.remove('last_date_of_month_8')

# for test data set
tel_churn_test.drop(columns=['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8'],inplace=True)

In [ ]:
# Removing duplicate ID columns
tel_churn.drop(['circle_id'],axis=1,inplace=True)
# for test data set
tel_churn_test.drop(['circle_id'],axis=1,inplace=True)

In [ ]:
import missingno as msno
msno.matrix(tel_churn,figsize=(20,10),fontsize=12,color=(0.42, 0.1, 0.05),sparkline=True,labels=True,label_rotation=90)
plt.show()

As it is evident from the chart there is some missing values in the data. We will try to find the pattern and fill the missing values accordingly. We are using MICE technique to fill the missing values.

In [ ]:
# Using MICE to impute missing values
imp = IterativeImputer(estimator=BayesianRidge(),max_iter=10, random_state=0)
# Fitting the imputer for each index in date columns
for i in date_cols:
    tel_churn[i] = imp.fit_transform(tel_churn[i].values.reshape(-1,1))

All the missing values are filled and thus can proceed with the next step of data analysis. As all columns are numeric we can proceed with scatter charts to find the correlation between the columns. One last step to remove unwanted columns from the data which have only one value.

In [ ]:
# Remove Columns with only one unique value
tel_churn = tel_churn.loc[:,tel_churn.nunique()!=1]
tel_churn.shape

## Deriving new features

In [ ]:
# Average recharge amount for June and July
tel_churn['avg_rech_amt_6_7']=((tel_churn['total_rech_amt_6']+tel_churn['total_rech_amt_7'])/2)
# for test data set
tel_churn_test['avg_rech_amt_6_7']=((tel_churn_test['total_rech_amt_6']+tel_churn_test['total_rech_amt_7'])/2)

In [ ]:
# Days user with company
tel_churn['days_stayed'] = tel_churn['date_of_last_rech_8'] - tel_churn['date_of_last_rech_6']
# for test data set
tel_churn_test['days_stayed'] = tel_churn_test['date_of_last_rech_8'] - tel_churn_test['date_of_last_rech_6']

In [ ]:
# Average 3g usage for June and July
tel_churn['avg_3g_6_7']=((tel_churn['vol_3g_mb_6']+tel_churn['vol_3g_mb_7'])/2)
# for test data set
tel_churn_test['avg_3g_6_7']=((tel_churn_test['vol_3g_mb_6']+tel_churn_test['vol_3g_mb_7'])/2)

In [ ]:
# Average 2g usage for June and July
tel_churn['avg_2g_6_7']=((tel_churn['vol_2g_mb_6']+tel_churn['vol_2g_mb_7'])/2)
# for test data set
tel_churn_test['avg_2g_6_7']=((tel_churn_test['vol_2g_mb_6']+tel_churn_test['vol_2g_mb_7'])/2)

In [ ]:
# Avergae of 6th and 7th month total usage
tel_churn['avg_total_6_7']=((tel_churn['total_og_mou_6']+tel_churn['total_og_mou_7'])/2)
# for test data set
tel_churn_test['avg_total_6_7']=((tel_churn_test['total_og_mou_6']+tel_churn_test['total_og_mou_7'])/2)

In [ ]:
# Avg. mou at action phase
# We are taking average because there are two months(7 and 8) in action phase
tel_churn['avg_mou_action'] = (tel_churn['total_og_mou_7'] + tel_churn['total_og_mou_8'] + tel_churn['total_ic_mou_7'] + tel_churn['total_ic_mou_8'])/2
# for test data set
tel_churn_test['avg_mou_action'] = (tel_churn_test['total_og_mou_7'] + tel_churn_test['total_og_mou_8'] + tel_churn_test['total_ic_mou_7'] + tel_churn_test['total_ic_mou_8'])/2

In [ ]:
# ARUP in action phase
tel_churn['avg_arpu_action'] = (tel_churn['arpu_7'] + tel_churn['arpu_8'])/2
# Difference of good and action phase ARPU
tel_churn['diff_arpu'] = tel_churn['avg_arpu_action'] - tel_churn['arpu_6']
# Checking whether the arpu has decreased on the action month
tel_churn['decrease_arpu_action'] = np.where(tel_churn['diff_arpu'] < 0, 1, 0)

# ARUP in action phase
tel_churn_test['avg_arpu_action'] = (tel_churn_test['arpu_7'] + tel_churn_test['arpu_8'])/2
# Difference of good and action phase ARPU
tel_churn_test['diff_arpu'] = tel_churn_test['avg_arpu_action'] - tel_churn_test['arpu_6']
# Checking whether the arpu has decreased on the action month
tel_churn_test['decrease_arpu_action'] = np.where(tel_churn_test['diff_arpu'] < 0, 1, 0)

## Filtering High-Value Customers

In [ ]:
# Filtering the customers based on average recharge amount
perc_6_7=tel_churn['avg_rech_amt_6_7'].quantile(0.70)
tel_churn=tel_churn[tel_churn['avg_rech_amt_6_7']>=perc_6_7]
tel_churn.shape

In [ ]:
# Column list with id and target variable
col_list_bar = tel_churn.columns.to_list()
col_list_bar.remove('churn_probability')
col_list_bar.remove('id')

## Outlier Treatment

In [ ]:
# Removing outlier using z-score method 
z = np.abs(stats.zscore(tel_churn[col_list_bar]))
tel_churn = tel_churn[(z < 5).all(axis=1)]
tel_churn.shape

In [ ]:
# Checking for outliers using boxplot in a 34 by 4 grid
fig, ax = plt.subplots(34, 4, figsize=(20, 60))
for variable, subplot in zip(col_list_bar, ax.flatten()):
    tel_churn.boxplot(column=variable, ax=subplot, vert=False)
plt.tight_layout()
plt.show()

As the outluers are present in large number and IQR method will result in loss of data we used z score method. As we are using a model to predict the churn we can use a model which is robust to outliers.

In [ ]:
# Creating copy of dataframe for Predictive model
tel_churn_pred = tel_churn.copy()

## Uni-variate Analysis

In [ ]:
# Checking for distribution using distplot in a 34 by 4 grid
fig, ax = plt.subplots(34, 4, figsize=(20, 100))
for variable, subplot in zip(col_list_bar, ax.flatten()):
    sns.distplot(tel_churn[variable], ax=subplot,kde=True, bins=50, hist_kws={'alpha': 1})
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.tight_layout()
plt.show()

## Bi-variate Analysis

In [ ]:
# Bivariate analysis of churn probability with other variables
fig, ax = plt.subplots(34, 4, figsize=(20, 100))
for variable, subplot in zip(col_list_bar, ax.flatten()):
    sns.boxplot(x='churn_probability', y=variable, data=tel_churn, ax=subplot)
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.tight_layout()
plt.show()

## Multi-variate Analysis

In [ ]:
# Heatmap with target variable
plt.figure(figsize=(20,30))
sns.heatmap(tel_churn_pred.corr()[['churn_probability']].sort_values(by='churn_probability',ascending=False),cmap='coolwarm',annot=True)
plt.show()

As we are using a model to predict the churn we can use a model which is robust to multi-collinearity and outliers.

## Model Building for Predicting Churn OBJECTIVE 1

In [ ]:
# train test split
X = tel_churn_pred.drop(['id','churn_probability'],axis=1)
y = tel_churn_pred['churn_probability']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=71, stratify=y)

In [ ]:
# Using Combined sampling to handle imbalanced dataset
smt = SMOTETomek(random_state=71, sampling_strategy=0.25, n_jobs=-1)
X_train, y_train = smt.fit_resample(X_train, y_train)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Standardization method
scaler = MinMaxScaler()
X_train[col_list_bar] = scaler.fit_transform(X_train[col_list_bar])
X_test[col_list_bar] = scaler.transform(X_test[col_list_bar])
tel_churn_test[col_list_bar] = scaler.transform(tel_churn_test[col_list_bar])

## Model Selection using RandomizedSearchCV

In [ ]:
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'mlogloss',
        'use_label_encoder':[ False,True],
    }

    # Fit the model
    optuna_model = XGBRFClassifier(**params, random_state=71,tree_method='gpu_hist', gpu_id=0)
    optuna_model.fit(X_train, y_train)

    # Make predictions
    y_pred = optuna_model.predict(X_test)

    return accuracy_score(y_test, y_pred)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

In [ ]:
print(f'Number of finished trials: {len(study.trials)}')
print('Best trial:')
trial = study.best_trial

print(f'  Value: {trial.value}')
print('  Params: ')

for key, value in trial.params.items():
    print(f'    {key}: {value}')

In [ ]:
params = trial.params
model = XGBRFClassifier(**params, random_state=71)
model.fit(X_train, y_train)

## Model Evaluation

In [ ]:
y_pred_train = model.predict(X_test)
print('Accuracy:',accuracy_score(y_test,y_pred_train))
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_train).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
print('Specificity:',specificity)

## Output for Solution

In [ ]:
# output the predicted the class for test data
output=pd.DataFrame({"id":tel_churn_test.id,"churn_probability":model.predict(tel_churn_test[X_train.columns])})
output.to_csv('submission.csv',index=False)

In [ ]:
# checking submission accuracy
solution=pd.read_csv("solution.csv")
solution.head()
accuracy_score(solution.churn_probability,output.churn_probability)

## Model Building for feature importance OBJECTIVE 2

In [ ]:
# Making copy for feature importance
tel_churn_features = tel_churn.copy()
X_feature=tel_churn_features[col_list_bar]
y_feature=tel_churn_features['churn_probability']

In [ ]:
# Using Combined sampling to handle imbalanced dataset
smt = SMOTETomek(random_state=40, sampling_strategy=0.2, n_jobs=-1)
X_feature, y_feature = smt.fit_resample(X_feature, y_feature)

In [ ]:
# Standardization method
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_feature[col_list_bar] = scaler.fit_transform(X_feature[col_list_bar])

In [ ]:
# extract feature importance
model = XGBClassifier(random_state=80)
model.fit(X_feature, y_feature)
importance = model.feature_importances_
# summarize feature importance
feature_imp=pd.DataFrame({"feature":X_feature.columns,"importance":importance/np.sum(importance)*100})
feature_imp.sort_values(by='importance',ascending=False,inplace=True)
feature_imp=feature_imp[feature_imp.importance>0]
feature_imp

In [ ]:
# Extracting features above 30% quantile for feature importance
feature_imp_50=feature_imp[feature_imp.importance>feature_imp.importance.quantile(0.3)]
feature_imp_50

In [ ]:
# Plotting feature importance
plt.figure(figsize=(20,10))
sns.barplot(x=feature_imp_50.feature,y=feature_imp_50.importance)
plt.xticks(rotation=90)
plt.show()

## Business Insights and Recommendations

In [ ]:
# Most important 20 features
feature_imp[feature_imp.importance>feature_imp.importance.quantile(0.8)]

In [ ]:
# Plotting top 20 features for churn and no churn using distplot
plt.figure(figsize=(20,20))
for i in range(1,21):
    plt.subplot(5,4,i)
    sns.distplot(tel_churn[tel_churn['churn_probability']==1][feature_imp[feature_imp.importance>feature_imp.importance.quantile(0.8)].feature.values[i-1]],label='Churn')
    sns.distplot(tel_churn[tel_churn['churn_probability']==0][feature_imp[feature_imp.importance>feature_imp.importance.quantile(0.8)].feature.values[i-1]],label='No Churn')
    plt.legend()
plt.show()

### Recomendations

- The compаny should focus on users with lower roаming outgoing cаlls in August by providing them with better plаns.
- Similаrly, the compаny should focus on users with lower roаming incoming cаlls in August by providing them with better plаns.
- Tаrget the customers, whose minutes of usаge of the incoming locаl cаlls аnd outgoing ISD cаlls аre less in the аction phаse (mostly in the month of August).
- Tаrget the customers, whose outgoing others chаrge in July аnd incoming others in August аre less.
- Also, the customers hаving vаlue-bаsed cost in the аction phаse increаsed аre more likely to churn thаn the other customers. Hence, these customers mаy be а good tаrget to provide offers.
- Customers, whose monthly 3G rechаrge in August is more, аre likely to be churned.
- Customers hаving to decreаse STD incoming minutes of usаge for operаtors T to fixed lines of T for the month of August аre more likely to churn.
- Customers decreаsing monthly 2g usаge for August аre most probаble to churn.
- Customers hаving decreаsing incoming minutes of usаge for operаtors T to fixed lines of T for August аre more likely to churn.
- roаm_og_mou_8 vаriаbles hаve positive coefficients. Thаt meаns thаt customers, whose roаming outgoing minutes of usаge аre increаsing аre more likely to churn.